# Retail Demo Store Experimentation Workshop - Overview

Welcome to the Retail Demo Store Experimentation Workshop. In this module we are going to add experimentation to our Retail Demo Store eCommerce website. This will allow us to experiment with different personalization approaches in the user interface. We take two different approaches adding experimention. First, we'll explore how to implement three different experimentation techniques from the ground up. This is the DIY path. Then we will explore how to use solutions from AWS partners [Amplitude](https://amplitude.com/) and [Optimizely](https://www.optimizely.com/).

The built-in experimentation techniques include the following.

- A/B testing
- Interleaving recommendation testing
- Multi-armed bandit testing

Each experimentation technique will be described in detail as well as how to evaluate results. Although these techniques will be used to measure different approaches to personalization, the framework presented can be easily used for other experimentation use-cases as well.

The AWS partner workshops Amplitude and Optimizely each use the A/B testing technique.

The workshops can be performed in any order.

Recommended Time: 15 minutes this notebook, ~1.5 hours total for all experiment workshops

## Prerequisites

Since this module uses the Retail Demo Store's Recommendation microservice to run experiments across variations that depend on the personalization features of the Retail Demo Store, it is assumed that you have either completed the [Personalization](../1-Personalization/Lab-01-(Core)-Introduction-and-data-preparation.ipynb) workshop or those resources have been pre-provisioned and configured in your AWS environment. If you are unsure and attending an AWS managed event such as a workshop or immersion day, check with your event lead.

## Experimentation Framework and Built-In Techniques

Before we can dive into the first experimentation technique, let's review the overall architecture (depicted below) and some of the code. Although this experimentation framework is implemented in the Python programming language, it can be easily ported to many other languages.

> All of the source code for the experimentation architecture described here can be found in the Retail Demo Store's [Recommendations](https://github.com/aws-samples/retail-demo-store/tree/master/src/recommendations) web service which is in a Git repository (AWS CodeCommit or GitHub) attached to AWS CodePipeline in your AWS environment.

![Experimentation Class Diagram](./images/experimentation-class-diagram.png)

### Variation

Let's start with **Variation**. In the context of this workshop, a **Varation** is a specific approach to providing product recommendations or ranking product lists for a user. As mentioned above, this framework can be used to test variations of any feature in the Retail Demo Store--we're just focusing on testing different approaches to making product recommendations or ranking products. A **Variation** represents the configuration information necessary to create a **Resolver** (described below) that is used to retrieve recommendations or rank a list of products. For example, for a variation backed by an Amazon Personalize campaign, the variation configuration would specify the Campaign ARN. For a variation backed by a "similar item" OpenSearch query in the Search service, the variation configuration would specify the host/IP for the Retail Demo Store Search microservice. And so on.

Here is the code for the [**Variation**](https://github.com/aws-samples/retail-demo-store/blob/master/src/recommendations/src/recommendations-service/experimentation/experiment.py) class.

```python
# from src/recommendations/src/recommendations-service/experimentation/experiment.py

class Variation:
    def __init__(self, **data):
        self.config = data
        self.resolver = ResolverFactory.get(**data)
```

There's not much to this class. It's holding the configuration information in `self.config` and creating the **Resolver** based on the configuration. The **ResolverFactory.get()** function takes the configuration for a resolver (it's type and initialization parameters) and returns the appropriate **Resolver** instance.

### Resolver

As mentioned above, a [**Resolver**](https://github.com/aws-samples/retail-demo-store/blob/master/src/recommendations/src/recommendations-service/experimentation/resolvers.py) retrieves product recommendations or ranks a product list for a particular **Variation**. Here is the code for the **Resolver** abstract base class.

```python
# from src/recommendations/src/recommendations-service/experimentation/resolvers.py

class Resolver(ABC):
    """ Abstract base class for all resolvers"""
    @abstractmethod
    def get_items(self, **kwargs):
        """ Returns recommended items for this resolver

        Arguments:
            Parameters needed by resolver to return recommendations
        
        Return: 
            List of dictionaries where each dictionary minimally includes an 'itemId' key representing a recommended item
        """
        pass
```

As you can see, a resolver only needs to implement the `get_items()` method. This method should return a list of dictionaries where each dictionary must have an `itemId` key with a value representing the ID for a recommended item. For this project, an item is a product. Here is a sample response with three items.

```python
[ 
    {
        'itemId': '56'
    }, 
    {
        'itemId': '7'
    }, 
    {
        'itemId': '13'
    }
]
```

There are six implementations of **Resolver** provided in this project. Four resolvers are used for the product recommendation and related item use-cases and two resolvers are used for the product list ranking use-case.

#### DefaultProductResolver

The [**DefaultProductResolver**](https://github.com/aws-samples/retail-demo-store/blob/master/src/recommendations/src/recommendations-service/experimentation/resolvers.py) will call the Retail Demo Store's Product microservice to retrieve a list of products in the same category of the current product. If the current product does not have an assigned category, the resolver will fallback to recommending featured products. Given the same product, all users will receive the same product recommendations. Therefore, this resolver does not provide personalized recommendations and is a good example of a very basic approach to making product suggestions.

Below is the relevant code from the `DefaultProductResolver.get_items` method. See the source repository for a complete listing.

```python
# from src/recommendations/src/recommendations-service/experimentation/resolvers.py

class DefaultProductResolver(Resolver):
    ...
    
    def get_items(self, **kwargs):
        ...
        category = None

        if product_id:
            # Lookup product to determine if it belongs to a category
            url = f'http://{self.products_service_host}/products/id/{product_id}'
            response = requests.get(url)

            if response.ok:
                category = response.json()['category']

        if category:
            # Product belongs to a category so get list of products in same category
            url = f'http://{self.products_service_host}/products/category/{category}'
            response = requests.get(url)
        else:
            # Product not specified or does not belong to a category so fallback to featured products
            url = f'http://{self.products_service_host}/products/featured'
            response = requests.get(url)

        if response.ok:
            # Create response making sure not to include current product
            for product in response.json():
                if product['id'] != product_id:
                    items.append({'itemId': str(product['id'])})

                    if len(items) >= num_results:
                        break
        else:
            raise Exception(f'Error calling products service: {response.status_code}: {response.reason}')

        return items
```

#### SearchSimilarProductsResolver

The [**SearchSimilarProductsResolver**](https://github.com/aws-samples/retail-demo-store/blob/master/src/recommendations/src/recommendations-service/experimentation/resolvers.py) leverages the Retail Demo Store's Search service to retrieve a ranked list of products similar to the current product. Internally the Search service uses the OpenSearch "[more like this](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-mlt-query.html)" query type. This can be considered a content filtering approach but still not personalized. Nevertheless, it is an improvement over **DefaultProductResolver** since items are ranked by relevance based on similarities across multiple product fields.

Below is the relevant code from the `SearchSimilarProductsResolver.get_items` method. See the source repository for a complete listing.

```python
# from src/recommendations/src/recommendations-service/experimentation/resolvers.py

class SearchSimilarProductsResolver(Resolver): 
    ...
    
    def get_items(self, **kwargs): 
        ...
        url = f'http://{self.search_service_host}/similar/products?productId={product_id}' 
        response = requests.get(url) 
 
        items = [] 
 
        if response.ok: 
            for product in response.json(): 
                items.append({'itemId': str(product['id'])}) 
 
                if len(items) >= num_results: 
                    break 
        else: 
            raise Exception(f'Error calling products service: {response.status_code}: {response.reason}') 
 
        return items 
```

#### PersonalizeRecommendationsResolver

Given a user, the [**PersonalizeRecommendationsResolver**](https://github.com/aws-samples/retail-demo-store/blob/master/src/recommendations/src/recommendations-service/experimentation/resolvers.py) will retrieve product recommendations from an Amazon Personalize campaign, such as the campaign created in the [Personalize workshop](../1-Personalization/Lab-1-Introduction-and-data-preparation.ipynb). Therefore, this resolver provides truly personalized recommendations.

Below is the relevant code from the `PersonalizeRecommendationsResolver.get_items` method. See the source repository for a complete listing.

```python
# from src/recommendations/src/recommendations-service/experimentation/resolvers.py

class PersonalizeRecommendationsResolver(Resolver):
    """ Provides recommendations from an Amazon Personalize campaign """
    __personalize_runtime = boto3.client('personalize-runtime')
    
    ...

    def get_items(self, **kwargs):
        user_id = kwargs.get('user_id')
        item_id = kwargs.get('product_id')

        if user_id is None and item_id is None:
            raise Exception('user_id or product_id is required')

        params = {}
        
        is_recommender = self.inference_arn.split(':')[5].startswith('recommender/')
        if is_recommender:
            params['recommenderArn'] = self.inference_arn
        else:
            params['campaignArn'] = self.inference_arn
        
        if user_id is not None:
            params['userId'] = user_id

        if item_id is not None:
            params['itemId'] = item_id

        ...

        response = PersonalizeRecommendationsResolver.__personalize_runtime.get_recommendations(**params)

        return response['itemList']
```

The **PersonalizeRankingResolver** resolver implementation is very similar but it calls the [get_personalized_ranking](https://docs.aws.amazon.com/personalize/latest/dg/getting-real-time-recommendations.html#rankings) function on Amazon Personalize instead.

#### HTTPResolver

Finally, the [**HTTPResolver**](https://github.com/aws-samples/retail-demo-store/blob/master/src/recommendations/src/recommendations-service/experimentation/resolvers.py) provides a generic example of how recommendations from an external recommendation service could be integrated into this experimentation framework. For instance, using this framework to evaluate, say, Amazon Personalize against an existing recommendation system.

See the source repository for a complete listing of **HTTPResolver**.

### Experiment Strategies

Pulling together the concepts of variations and resolvers described above, there are three experimentation strategies implemented in this project. Each of the experimentation techniques are implemented as a subclass of the [**Experiment**](https://github.com/aws-samples/retail-demo-store/blob/master/src/recommendations/src/recommendations-service/experimentation/experiment.py) abstract base class.

#### Experiment

The [**Experiment**](https://github.com/aws-samples/retail-demo-store/blob/master/src/recommendations/src/recommendations-service/experimentation/experiment.py) class defines the contract required for all subclasses as well as provides some utility methods for creating correlation identifiers and tracking exposure and outcome events. An exposure represents a user being exposed to an experiment variation and an outcome represents a user interacting with an exposure item (i.e. clicking on a recommended product).

Let's take a closer look at relevant parts of the **Experiment** base class.

```python
# from src/recommendations/src/recommendations-service/experimentation/experiment.py

class Experiment(ABC):
    ...
    def __init__(self, table, **data):
        self._table = table
        self.id = data['id']
        self.feature = data['feature']
        self.name = data['name']
        self.status = data['status']
        self.type = data['type']

        self.variations = []

        for v in data['variations']:
            self.variations.append(Variation(**v))
    
    @abstractmethod
    def get_items(self, user_id, current_item_id = None, item_list = None, num_results = 10, tracker = None):
        """ For a given user, returns item recommendations for this experiment along with experiment tracking/correlation information """
        pass

    def track_conversion(self, correlation_id):
        """ Call this method to track a conversion/outcome for an experiment """
        correlation_bits = correlation_id.split('~')
        variation_index = int(correlation_bits[2])

        if variation_index < 0 or variation_index >= len(self.variations):
            raise Exception('variation_index is out of bounds')

        return self._increment_convert_count(variation_index)

    def _increment_exposure_count(self, variation, count = 1):
        """ Call this method when a user is exposed to a variation of an experiment """
        return self.__increment_variation_count('exposures', variation, count)
```

The initialization/constructor method captures configuration information for the experiment. It also creates the **Variation** instances for the experiment that will be used to retrieve recommendations from each variation's resolver.

The abstract `get_items()` method is implemented by each experiment subclass to return item recommendations according to its specific implementation. For example, for the A/B experiment the user will be assigned to the control or variation group and then receive recommendations from the corresponding resolver. 

Finally, exposure and conversion/outcome events are tracked using the `_increment_exposure_count()` and `track_conversion()` methods. You will see how these methods are used by each experiment type within their respective exercise notebooks.

#### ABExperiment

The [**A/B Experiment**](https://github.com/aws-samples/retail-demo-store/blob/master/src/recommendations/src/recommendations-service/experimentation/experiment_ab.py) class provides an A/B testing implementation such that given a `user_id` it will randomly and consistently assign a user to receive recommendations from one of two or more variations.

#### InterleavingExperiment

The [**Interleaving Experiment**](https://github.com/aws-samples/retail-demo-store/blob/master/src/recommendations/src/recommendations-service/experimentation/experiment_interleaving.py) class provides an implementation of interleaving or blending recommendations from two or more variations for each user. There are two interleaving method implementations provided. The **Balanced Method** balances selections from each variation in an alternating fashion. The **Team Draft Method** uses the analogy of team captains that are randomly selected for each round to select their next top "player" when building the interleaved results.

#### MultiArmedBanditExperiment

Finally, the [**Multi-Armed Bandit Experiment**](https://github.com/aws-samples/retail-demo-store/blob/master/src/recommendations/src/recommendations-service/experimentation/experiment_mab.py) class provides an implementation using a Bayesian technique to continually evaluate the performance of two or more variations such that the best performing variation is selected most often (exploitation) while the other variations are occasionally selected should their performance change over time (exploration). 

Each of these strategies will be discussed in detail with exercises in this workshop which are implemented as separate notebooks.

## Experiment Strategy Datastore

As mentioned above, the experimentation framework described in this workshop is completely contained in the Retail Demo Store's Recommendation microservice. The microservice determines which experiments are active and their configuration by querying the experiment strategy table in DynamoDB. The table was automatically created by the Retail Demo Store CloudFormation templates. Since the table name is dynamically created at deployment time, the table name will be stored in and loaded from an AWS Systems Manager parameter.

The Retail Demo Store supports running multiple experiments concurrently but for this module we will run an experiment for each technique sequentially to make it easier to interpret results. Experiment configurations are stored in a DynamoDB table where each item in the table represents an experiment and has the following fields.

- **id** - Uniquely identifies an experiment (UUID).
- **feature** - Identifies the Retail Demo Store feature where the experiment should be applied. Each testable feature in the UI has its own unique name. For example, recommended products panel on the home page has a feature name of `home_product_recs`. The related products panel on the product detail page has a feature name of `product_detail_related`. And so on. For this workshop we are running experiments for the product recommendations on the home page so the feature we'll be working with is `home_product_recs`. 
- **name** - The name of the experiment. The name is determined by the person running the experiment and should be short but descriptive. It will be used in the UI for demo purposes and when logging events for experiment outcome tracking.
- **type** - The type of test (`ab` for an A/B test, `interleaving` for interleaved recommendations, or `mab` for multi-armed bandit test)
- **status** - The status of the experiment (`ACTIVE`, `EXPIRED`, or `PENDING`). Only active experiments will be rendered in the UI.
- **variations** - List of configurations representing variations applicable for the feature and type. For example, for A/B tests of the `home_product_recs` feature, the `variations` can be two Amazon Personalize campaign ARNs (variation type `personalize`) or a single Personalize campaign ARN and the default product behavior (variation type `product`). Really any combination of variations.

Here is an example of an experiment strategy (i.e. DynamoDB item) for an A/B experiment that is testing the default product resolver (first variation or control group) and an Amazon Personalize campaign (second variation).

```javascript
{
  "id": "17077add918e40e4ab1abacb860ea0ad",
  "feature": "home_product_recs",
  "name": "home_personalize_ab",
  "type": "ab",
  "status": "ACTIVE",
  "variations": [
    {
      "type": "product"
    },
    {
      "type": "personalize",
      "campaign_arn": "arn:aws:personalize:us-west-2:[redacted]:campaign/retaildemostore-product-personalization"
    }
  ]
}
```

### Experiment Response

Let's bring everything together by walking through how experiments for product recommendations are surfaced on the Retail Demo Store's home page.

1. A user signs in to the Retail Demo Store's web site and browses to the home page.
2. The home page makes an API request to the Retail Demo Store's [Recommendations](https://github.com/aws-samples/retail-demo-store/tree/master/src/recommendations) microservice (`/recommendations` endpoint) passing the current user's ID and the feature name of `home_product_recs`.
3. The Recommendations service calls `ExperimentManager().get_active('home_product_recs')` to get the active **Experiment** for the `home_product_recs` feature.
4. The Recommendations service calls `Experiment.get_items()` method passing the `user_id` to get the product recommendations for the user based on the currently active experiment.
5. The Recommendations service returns the product recommendations response to the home page as JSON.
6. The home page renders the product recommendations in the "Inspired by your shopping trends" section of the page.

Below is an example of the response from the Recommendations service for an A/B experiment. The `itemId` is the ID for the recommended product and the `experiment` dictionary includes information on the experiment and variation. 

```javascript
[
    {
        "itemId": "2",
        "experiment": {
            "id": "d56619ad367f463688f506eefc0d08ca",
            "feature": "home_product_recs",
            "name": "home_personalize_ab",
            "type": "ab",
            "variationIndex": 0,
            "resultRank": 1,
            "correlationId": "d56619ad367f463688f506eefc0d08ca-42-0-1"
        }
    },
    {
        "itemId": "10",
        "experiment": {
            "id": "d56619ad367f463688f506eefc0d08ca",
            "feature": "home_product_recs",
            "name": "home_personalize_ab",
            "type": "ab",
            "variationIndex": 0,
            "resultRank": 2,
            "correlationId": "d56619ad367f463688f506eefc0d08ca-42-0-2"
        }
    },
    {
        "itemId": "18",
        "experiment": {
            "id": "d56619ad367f463688f506eefc0d08ca",
            "feature": "home_product_recs",
            "name": "home_personalize_ab",
            "type": "ab",
            "variationIndex": 0,
            "resultRank": 3,
            "correlationId": "d56619ad367f463688f506eefc0d08ca-42-0-3"
        }
    }
]
```

> As you can see, we're returning a lot of extra information on the experiment and variation for each recommended item. This is being done so that the Retail Demo Store UI can annotate recommended items on the page so we can visualize the effects of the experiments. In a production scenario, the `experiment.correlationId` is all that is needed to correlate user behavior with experiments.

## User Interface Treatment

To make it easier to verify what's going on with your active experiments, the Retail Demo Store user interface will annotate content from experiments and variations received from the Recommendations service (see the red boxes in the following screenshot of the home page with an active experiement). While working through the exercises in this workshop you can open the Retail Demo Store's Web UI in another browser tab/window and verify that your experiment is active and working as expected. By default the Retail Demo Store UI displays information on active tests to make it easier to demo and debug. Of course, in a production deployment this information would be hidden from the user.

> To find the URL of the Retail Demo Store Web UI for your deployment, browse to AWS CloudFormation in your AWS environment, find the stack for your Retail Demo Store deployment, click on Outputs, and locate the "WebURL" key. Open this URL in a separate browser tab/window.

![Experiment UI Treatment](./images/ui-treatment.png)


## Next Steps

With a foundational understanding of the experimentation framework now established, let's work through detailed exercises for each experiment type. Open the next notebook, **[3.2-AB-Experiment](./3.2-AB-Experiment.ipynb)**, in this workshop to step through running and A/B experiment.